# Regularización

## Regularización Paramétrica

- Normas L1 y L2

## Conceptos básicos en Machine Learning

Se puede dividir en 4 grandes pasos:

1. Dividir:
    - Corresponde en:
        1. Dividir en vectores predictivos y vector objetivo
        2. Dividir en datos de entrenamiento y datos para testear el modelo
        - Por lo tanto, se generan 4 subconjuntos de datos.
2. Entrenar:
    - Corresponde a definir que funcion matemática/algoritmo queremos utilizar para hacer nuestro modelo predictivo
    - Con los datos de entrenamiento, se entrena a nuestro modelo

3. Predicir:
    - Una vez entrenado nuestro modelo, procedemos a hacer predicciones
    - En este caso, se utilizan los vectores predictivos del set de testeo para hacer las predicciones

4. Evaluar:
    - Cuando ya realizamos nuestra predicciones, procedemos a hacer la evaluación de nuestro modelo. Esto se realiza contrastando el vector objetivo del set de testeo, y las predicciones realizadas previamente
    - Para variables continuas, se puede utilizar el Mean Squared Error
    - Para variables categoricas, se puede utilizar una matriz de confusion

## Subajuste/Sobreajuste

Son fenómenos que se pueden observar al realizar un modelo predictivo.

- **Subajuste:** Ocurre cuando nuestro modelo es demasiado rígido, y se le hace difícil explicar las distintas variaciones que existen en los datos de entrenamiento. Lo bueno es que esto hace que sea mas generalizable a datos nuevos. Tiene un alto sesgo (rígido), explica poco a los datos de entrenamiento y tiene una alta chance de explicar mejor un nuevo set de datos.

- **Sobreajuste:** Ocurre cuando nuestro modelo se ajustó demasiado a nuestros datos de entrenamiento. Esto hace que sea muy malo para predecir nuevos datos. Tiene una alta variabilidad (demasiado flexible), explica muy bien los datos de entrenamiento, pero tiene menores chanches de explicar un nuevo set de datos

![Fitting](https://media.geeksforgeeks.org/wp-content/cdn-uploads/20190523171258/overfitting_2.png)


# Elementos básicos de la regularización

Tomando como punto de partida a la regresión lineal.

- Para obtener la linea recta se utiliza el método de los mínimos cuadrados
- Esta forma permite que las estimaciones sean insesgadas y óptimas

Pero, por qué regularizar?

1. Puede que hayan parámetros que tengan un peso exagerado en la estimación
2. Puede hacer que la estimación del modelo sea mucho más óptima (sobretodo cuando hay muchas variable predictivas)
3. Permite seleccionar los mejores parámetros
4. Permite evaluar insesgadamente cuales son las mejores opciones para hacer un modelo

### Norma L1

Sintentiza la distancia entre dos vectores mediante la norma absoluta. Se conoce como Lasso.

### Norma L2

Sintentiza la distancia entre dos vectores mediante la norma euclídea. Se conoce como Ridge.

# Datos

Es un conjunto de datos que posee los precios de casas junto a sus atributos (Numero de Baños,
Inmueble, Cantidad de personas, etc...). En este caso, se quiere predecir, ocupando diversos
atributos, el precio de la casa.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from cv_error import *
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 6)
plt.style.use('seaborn-darkgrid')